# 讀取檔案

In [ ]:
# 讀取 kiara 訓練資料 (colab才需要 本機可以直接讀取資料夾)
!gdown "1iHLoYCZiLiZKfDrIlzdQ3IpiBxRrtw1n" --output kiara.zip
!unzip kiara.zip -d kiara_data

In [ ]:
# 讀取 kronii 訓練資料 (colab才需要 本機可以直接讀取資料夾)
!gdown "1JqO2aO3aDZyj4QqGZcdA1TeZrsLtBrqS" --output kronii.zip
!unzip kronii.zip -d kronii_data

# 製作圖片資料 (問題)

In [ ]:
# import
import glob
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from PIL import Image

# 空list 準備放 kiara 資料
kiara_list = []

# 使用 glob 讀取指定路徑中的所有檔案
# 本機的話 請指定放置圖片的資料夾位置
for img in glob.glob("/content/kiara_data/Kiara_*.jpg"):
    vtuber_img = plt.imread(img)                     # 讀取圖片
    vtuber_img = Image.fromarray(vtuber_img)               # 轉為PIL pillow 物件
    vtuber_img_resized = np.array(vtuber_img.resize((150,150)))     # pillow物件 可以調整大小 resize -> resize後 轉np.array
    kiara_list.append(vtuber_img_resized)                 # 放入list中

kiara_array = np.array(kiara_list) # list轉np.array

In [ ]:
kiara_train = kiara_array[:1000]     # x_train
kiara_test = kiara_array[1500:1600]   # x_test

In [ ]:
kronii_list = []

for img in glob.glob("/content/kronii_data/Kronii_*.jpg"):
    vtuber_img = plt.imread(img)
    vtuber_img = Image.fromarray(vtuber_img)
    vtuber_img_resized = np.array(vtuber_img.resize((150,150)))
    kronii_list.append(vtuber_img_resized)

kronii_array = np.array(kronii_list)

In [ ]:
kronii_train = kronii_array[:1000]     # x_train
kronii_test = kronii_array[1500:1600]   # x_test

In [ ]:
# 整合兩位vtuber的資料集
x_train = np.concatenate([kiara_train, kronii_train], axis=0)
x_test = np.concatenate([kiara_test, kronii_test], axis=0)

# 製作答案資料  (答案)

In [ ]:
# x_train 有2000張圖片 所以需要準備2000個答案
# x_test 有200張圖片  ..
x_train.shape, x_test.shape

In [ ]:
# 
y_train=[]
for i in range(1000):
    y_train.append("0")

for i in range(1000):
    y_train.append("1")

y_train = np.array(y_train)

In [ ]:
#
y_test=[]
for i in range(100):
    y_test.append("0")

for i in range(100):
    y_test.append("1")

y_test = np.array(y_test)

In [ ]:
# 檢查 x_train, y_train, x_test, y_test
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((2000, 150, 150, 3), (2000,), (200, 150, 150, 3), (200,))

# 資料預處理

In [ ]:
# 資料預處理- normalized
x_train_normalized = x_train / 255
x_test_normalized = x_test / 255

In [ ]:
# 資料預處理- one hot encoding
from keras.utils.np_utils import to_categorical


y_train_categorical = to_categorical(y_train)
y_test_categorical = to_categorical(y_test)

# 建立模型

In [ ]:
# 建立模型
from keras.models import Sequential
from keras.layers import Dense 
from keras.layers import Flatten
from keras.layers import Conv2D, MaxPooling2D


# 建立
cnn = Sequential()

# 第一層
cnn.add(Conv2D(filters=32, 
               kernel_size=(3, 3), 
               input_shape=(150, 150, 3), 
               padding="same",
               activation="relu"))
cnn.add(MaxPooling2D(pool_size=(2, 2)))

# 第二層
cnn.add(Conv2D(filters=64,
               kernel_size=(3, 3),
               padding="same",
               activation="relu"))
cnn.add(MaxPooling2D(pool_size=(2, 2)))

# 第三層
cnn.add(Conv2D(filters=128,
               kernel_size=(3, 3),
               padding="same",
               activation="relu"))
cnn.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten
cnn.add(Flatten())

# 第一層 
cnn.add(Dense(units=4096, activation="relu"))

# 第二層
cnn.add(Dense(units=1024, activation="relu"))

# 第三層
cnn.add(Dense(units=256, activation="relu"))

# 第四層
cnn.add(Dense(units=64, activation="relu"))

# 第五層
cnn.add(Dense(units=2, activation="softmax"))

# summary
cnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 150, 150, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 75, 75, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 75, 75, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 37, 37, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 37, 37, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 18, 18, 128)      0

# compile 

In [ ]:
# loss訓練方式

cnn.compile(loss="categorical_crossentropy",
            metrics=["accuracy"],
            optimizer="adam")

# fit

In [ ]:
# fit

cnn.fit(x_train_normalized, y_train_categorical,
        epochs=10,
        batch_size=200,
        validation_split=0.1,
        verbose=2)

Epoch 1/10
9/9 - 18s - loss: 1.6821 - accuracy: 0.6283 - val_loss: 0.2149 - val_accuracy: 0.9900 - 18s/epoch - 2s/step
Epoch 2/10
9/9 - 2s - loss: 0.0290 - accuracy: 0.9983 - val_loss: 1.0449e-05 - val_accuracy: 1.0000 - 2s/epoch - 207ms/step
Epoch 3/10
9/9 - 2s - loss: 0.9874 - accuracy: 0.9200 - val_loss: 0.0301 - val_accuracy: 0.9950 - 2s/epoch - 261ms/step
Epoch 4/10
9/9 - 2s - loss: 0.1059 - accuracy: 0.9772 - val_loss: 0.0000e+00 - val_accuracy: 1.0000 - 2s/epoch - 210ms/step
Epoch 5/10
9/9 - 2s - loss: 2.1457e-06 - accuracy: 1.0000 - val_loss: 7.5216e-07 - val_accuracy: 1.0000 - 2s/epoch - 202ms/step
Epoch 6/10
9/9 - 2s - loss: 0.0015 - accuracy: 0.9994 - val_loss: 1.2437e-05 - val_accuracy: 1.0000 - 2s/epoch - 202ms/step
Epoch 7/10
9/9 - 2s - loss: 5.7048e-05 - accuracy: 1.0000 - val_loss: 6.8005e-07 - val_accuracy: 1.0000 - 2s/epoch - 202ms/step
Epoch 8/10
9/9 - 2s - loss: 1.4389e-05 - accuracy: 1.0000 - val_loss: 2.6166e-07 - val_accuracy: 1.0000 - 2s/epoch - 208ms/step
Epoch

# evaluate

In [ ]:
# evaluate

final_loss, final_accuracy = cnn.evaluate(x_test_normalized, y_test_categorical)
print(f"最終loss值: {final_loss}, 最終正確率: {final_accuracy}")

7/7 [==============================] - 1s 29ms/step - loss: 8.8215e-07 - accuracy: 1.0000
最終loss值: 8.821466508379672e-07, 最終正確率: 1.0


# 混淆矩陣 
1. (跑不出來, 原因待查)

In [ ]:
# 混淆矩陣
from sklearn.metrics import confusion_matrix
import numpy as np
import pandas as pd


pre = np.argmax(cnn.predict(x_test_normalized), axis=1)
# 無法畫出矩陣 錯誤不明

7/7 [==============================] - 0s 13ms/step


# 畫圖 & 檢視正確值 與 預測值

In [ ]:
# 畫出200張圖
plt.figure(figsize=(20,25))

for (index, img) in enumerate(x_test):
    plt.subplot(10, 21, index+1)
    plt.axis("off")
    plt.title(f"predict:{pre[index]}\ntrue:{y_test[index]}")
    plt.imshow(img)

Output hidden; open in https://colab.research.google.com to view.

# 上傳其他圖片測試模型

In [ ]:
# 上傳圖片測試
from tensorflow.keras.preprocessing import image

img_path = '/content/test1.jpg'
img = image.load_img(img_path, target_size=(150, 150))
x = image.img_to_array(img)
x = x / 255.0
x = np.expand_dims(x, axis=0)

prediction = cnn.predict(x)

predicted_class_index = np.argmax(prediction)

anslist=["kiara", "kronii"]
print(anslist[predicted_class_index])

1/1 [==============================] - 0s 35ms/step
kronii
